In [1]:
import os
root = os.getcwd().rsplit('/',1)[0]
vocab_path = '{0:s}/etc/logset.ttl'.format(root)
dict_path = '{0:s}/etc/dict.ttl'.format(root)
print(vocab_path)
print(dict_path)

/global/u1/s/sleak/Monitoring/Resilience/LogSet/etc/logset.ttl
/global/u1/s/sleak/Monitoring/Resilience/LogSet/etc/dict.ttl


In [2]:
import sys
sys.path.append('/global/homes/s/sleak/.local/cori/3.6-anaconda-4.4/lib/python3.6/site-packages')
import rdflib
#graph = rdflib.ConjunctiveGraph().parse(vocab_path, format='turtle')
gvocab = rdflib.ConjunctiveGraph().parse(vocab_path, format='turtle')

# now add the data dictionary:
#graph.parse(dict_path, format='turtle')
gdict = rdflib.ConjunctiveGraph().parse(dict_path, format='turtle')
#gdict = rdflib.ConjunctiveGraph().parse(dict_path)
graph = gvocab + gdict

print(graph.serialize(format='n3').decode('ascii'))
#print(gdict.serialize(format='n3').decode('ascii'))
#print(graph.identifier)
#print(gvocab.identifier)
#print(gdict.identifier)



@prefix : <http://portal.nersc.gov/project/mpccc/sleak/resilience/datasets/dict#> .
@prefix dcat: <http://www.w3.org/ns/dcat#> .
@prefix dct: <http://purl.org/dc/terms/> .
@prefix default1: <http://portal.nersc.gov/project/mpccc/sleak/resilience/datasets/logset#> .
@prefix logset: <http://portal.nersc.gov/project/mpccc/sleak/resilience/datasets/logset#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

:annotationDB a logset:LogSeries ;
    logset:logFormat :sqlite3 .

:annotations a logset:InfoType .

:binaryFile a logset:LogFormatType ;
    dcat:mediaType "application/octet-stream" .

:dateYYYYMMDD a logset:FilenamePattern ;
    logset:regex "(?P<date>(?P<year>20[0-9]{2})(?P<month>0[1-9]|1[0-2])(?P<day>[0-3][0-

In [3]:
gindex = rdflib.ConjunctiveGraph().parse('{0:s}/examples/index.ttl'.format(root), format='turtle')
#graph.parse('{0:s}/examples/index.ttl'.format(root), format='turtle')
graph = (gvocab + gdict) + gindex
print(graph.serialize(format='n3').decode('ascii'))
#for s,p,o in graph:
for s,p,o in gindex:    
    print ((s,p,o))
    

@prefix : <http://portal.nersc.gov/project/mpccc/sleak/resilience/datasets/dict#> .
@prefix cray: <file:///global/u1/s/sleak/Monitoring/Resilience/LogSet/examples/cray-dict#> .
@prefix dcat: <http://www.w3.org/ns/dcat#> .
@prefix dct: <http://purl.org/dc/terms/> .
@prefix default1: <file:///global/u1/s/sleak/Monitoring/Resilience/LogSet/examples/index#> .
@prefix default11: <http://portal.nersc.gov/project/mpccc/sleak/resilience/datasets/logset#> .
@prefix dict: <http://portal.nersc.gov/project/mpccc/sleak/resilience/datasets/dict#> .
@prefix logset: <http://portal.nersc.gov/project/mpccc/sleak/resilience/datasets/logset#> .
@prefix nersc: <file:///global/u1/s/sleak/Monitoring/Resilience/LogSet/examples/nersc#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .
@prefix vcard: <http://www.w3.org/2006/vcard/ns#> .
@pref

In [7]:
query = '''SELECT ?id ?desc
WHERE {
?id a logset:LogSet .
?id dct:description ?desc .
}'''
#print(query[21])
for row in graph.query(query):
    print(row)

    print("strings not uris:")
for subject in graph.subjects(rdflib.RDF.type, 'http://portal.nersc.gov/project/mpccc/sleak/resilience/datasets/logset#LogSeries'):
    print(subject)
print("uris")
for subject in graph.subjects(rdflib.RDF.type, rdflib.term.URIRef('http://portal.nersc.gov/project/mpccc/sleak/resilience/datasets/logset#LogSeries')):
    print(subject)

(rdflib.term.URIRef('file:///global/u1/s/sleak/Monitoring/Resilience/LogSet/examples/index#'), rdflib.term.Literal('a partial sample of cori log data for testing logset tools'))
strings not uris:
uris
http://portal.nersc.gov/project/mpccc/sleak/resilience/datasets/dict#annotationDB


In [5]:
query = '''SELECT ?id ?path
WHERE {
?id logset:isLogOf dict:console_logfile .
?id logset:path ?path .
} '''
print("quering the whole graph:")
for row in graph.query(query):
    print(row)
    
# if I don't explicitly include the logset.ttl in the graph:
print("querying only gindex:")
# (oh - actually everything needed is in the index.ttl)
for row in gindex.query(query):
    print(row)


quering the whole graph:
querying only gindex:


In [7]:
query = '''SELECT ?id ?path
WHERE {
?id a ?type .
?type rdfs:subClassOf* logset:ConcreteLog . 
OPTIONAL { ?id logset:path ?path }
} '''
query = '''SELECT ?id ?path
WHERE {
?id a ?type .
?type rdfs:subClassOf* dcat:Distribution . 
OPTIONAL { ?id logset:path ?path }
} '''
# :ConcreteLog is defined in logset.ttl .. so if our graph includes that (local file) we get:
print ("with full graph:")
for row in graph.query(query):
    print(row)
    
# on the other hand, if we only query the local gindex graph:
print ("with gindex only:")
for row in gindex.query(query):
    print(row)


with full graph:
with gindex only:


In [7]:
# so to assemble the full graph, I need to explicitly parse the vocab and data dictionary:
# (mostly useful for catalog, as it will find subgraphs from there)

# can i parse a remote graph at all?
#g1 = rdflib.Graph().parse('https://www.w3.org/ns/dcat.ttl')
#print (g1)
# yes! but providing a format might be necessary
#gg = rdflib.Graph().parse('https://raw.githubusercontent.com/NERSC/LogSet/master/etc/logset.ttl', format='turtle')
gindex = rdflib.ConjunctiveGraph().parse('{0:s}/examples/index.ttl'.format(root), format='turtle')
##for s,p,o in gindex:    
##    print ((s,p,o))
namespaces=dict([n for n in gindex.namespaces()])
print(namespaces['logset'])
for n in gindex.namespaces():
    print (n)
#print(dir(rdflib.term.URIRef))
#print(rdflib.term.URIRef.__class__)
g = gindex
print ("g now has {0} statements".format(len(g)))
for uri in [n[1] for n in gindex.namespaces() if n[0] in ('logset','dict')]:
    print(str(uri.n3()))
    print(str(uri)[:-1])
    gsource=str(uri)[:-1] + '.ttl'   # needs extension or 404!
    g += rdflib.ConjunctiveGraph().parse(gsource, format='turtle')
    print ("g now has {0} statements".format(len(g)))
    #summary = g.serialize(format='n3').decode('ascii')
    #print(summary.splitlines()[:10])
    #print(g.serialize(format='n3').decode('ascii'))

# now what if I run that same query:
query = '''SELECT ?id ?path
WHERE {
?id a ?type .
?type rdfs:subClassOf* dcat:Distribution . 
OPTIONAL { ?id logset:path ?path }
} '''
for row in g.query(query):
    print(row)
    
print("logseries:")
for subject in graph.subjects(rdflib.RDF.type, rdflib.term.URIRef('http://portal.nersc.gov/project/mpccc/sleak/resilience/datasets/logset#LogSeries')):
    print(subject)

http://portal.nersc.gov/project/mpccc/sleak/resilience/datasets/logset#
('xml', rdflib.term.URIRef('http://www.w3.org/XML/1998/namespace'))
('rdf', rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#'))
('rdfs', rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#'))
('xsd', rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#'))
('dct', rdflib.term.URIRef('http://purl.org/dc/terms/'))
('dcat', rdflib.term.URIRef('http://www.w3.org/ns/dcat#'))
('vcard', rdflib.term.URIRef('http://www.w3.org/2006/vcard/ns#'))
('logset', rdflib.term.URIRef('http://portal.nersc.gov/project/mpccc/sleak/resilience/datasets/logset#'))
('dict', rdflib.term.URIRef('http://portal.nersc.gov/project/mpccc/sleak/resilience/datasets/dict#'))
('cray', rdflib.term.URIRef('file:///global/u1/s/sleak/Monitoring/Resilience/LogSet/examples/cray-dict#'))
('nersc', rdflib.term.URIRef('file:///global/u1/s/sleak/Monitoring/Resilience/LogSet/examples/nersc#'))
('', rdflib.term.URIRef('file:///global/u1/s/slea

In [9]:
query = '''SELECT ?id ?path
WHERE {
?id logset:isLogOf dict:console_logfile .
?id logset:path ?path .
} '''
# now we can find the datasources in the loaded graph
query2 = '''SELECT ?id ?path ?time
WHERE {
?id a ?type .
?type rdfs:subClassOf* logset:ConcreteLog . 
?id dct:temporal ?time .
OPTIONAL { ?id dcat:downloadURL ?path }
} '''
#query3 = '''SELECT ?id ?type ?base
#WHERE {
#?id a ?type .
#OPTIONAL { ?type rdfs:subClassOf ?base }
#}'''
for row in g.query(query2):
    print(row)

(rdflib.term.URIRef('file:///global/u1/s/sleak/Monitoring/Resilience/LogSet/examples/index#console-20170907'), rdflib.term.Literal('p0-20170906t151820/console-20170907'), rdflib.term.BNode('ub4bL63C18'))
(rdflib.term.URIRef('file:///global/u1/s/sleak/Monitoring/Resilience/LogSet/examples/index#console-20170906'), rdflib.term.Literal('p0-20170906t151820/console-20170906'), rdflib.term.BNode('ub4bL52C18'))
(rdflib.term.URIRef('file:///global/u1/s/sleak/Monitoring/Resilience/LogSet/examples/index#messages-20170907'), rdflib.term.Literal('p0-20170906t151820/messages-20170907'), rdflib.term.BNode('ub4bL107C18'))
(rdflib.term.URIRef('file:///global/u1/s/sleak/Monitoring/Resilience/LogSet/examples/index#consumer-20170906'), rdflib.term.Literal('p0-20170906t151820/consumer-20170906'), rdflib.term.BNode('ub4bL75C18'))
(rdflib.term.URIRef('file:///global/u1/s/sleak/Monitoring/Resilience/LogSet/examples/index#messages-20170906'), rdflib.term.Literal('p0-20170906t151820/messages-20170906'), rdflib

In [12]:
query = '''SELECT ?id ?path
WHERE {
?id logset:isInstanceOf [ dict:console_logfile . 
?id dcat:downloadURL ?path .
} '''
query = '''SELECT ?id
WHERE {
?id a logset:LogSeries .
}
'''
query = '''SELECT ?id ?path
WHERE {
?id logset:isInstanceOf [a logset:LogSeries] .
?id dcat:downloadURL ?path .
}
'''
for row in g.query(query):
    print(row)

In [10]:
query = '''SELECT ?series ?filepattern
WHERE {
?series a logset:LogSeries .
?series logset:logFormatInfo ?filepattern .
}
'''
for row in g.query(query):
    print(row)
# it seems graphs don't do regex filtering of literals well, so best is 
# to collect the logFormatInfos for each series, build a dictionary of 
# lists-of-values, and work with that

(rdflib.term.URIRef('https://raw.githubusercontent.com/NERSC/LogSet/master/etc/dict#consumer_logfile'), rdflib.term.Literal('filepattern=consumer'))
(rdflib.term.URIRef('https://raw.githubusercontent.com/NERSC/LogSet/master/etc/dict#consumer_logfile'), rdflib.term.Literal('filepattern=consumer-<dateYYYYMMDD>'))
(rdflib.term.URIRef('https://raw.githubusercontent.com/NERSC/LogSet/master/etc/dict#messages_logfile'), rdflib.term.Literal('filepattern=messages'))
(rdflib.term.URIRef('https://raw.githubusercontent.com/NERSC/LogSet/master/etc/dict#messages_logfile'), rdflib.term.Literal('ts_word=1'))
(rdflib.term.URIRef('https://raw.githubusercontent.com/NERSC/LogSet/master/etc/dict#messages_logfile'), rdflib.term.Literal('filepattern=messages-<dateYYYYMMDD>'))
(rdflib.term.URIRef('https://raw.githubusercontent.com/NERSC/LogSet/master/etc/dict#messages_logfile'), rdflib.term.Literal('part_word=2'))
(rdflib.term.URIRef('https://raw.githubusercontent.com/NERSC/LogSet/master/etc/dict#console_logf

In [11]:
# next: querying on datetime fields:
query = '''SELECT ?id ?path ?start
WHERE {
?id logset:isInstanceOf [a logset:LogSeries] .
?id dcat:downloadURL ?path .
?id dct:temporal/logset:startDate ?start .
FILTER (?start >= "2017-09-07-07:00"^^xsd:dateTime)
}
'''
# FILTER (?time = "2017-09-07"^^xsd:dateTime)
# this doesn't work:
# ?id dct:temporal [a dct:PeriodOfTime ; logset:startDate ?start ; OPTIONAL { logset:endDate ?end } ] .
for row in g.query(query):
    print(row)

(rdflib.term.BNode('f1454b393d3b34d8fbf97b594baecf658b5'), rdflib.term.Literal('p0-20170906t151820/consumer-20170907'), rdflib.term.Literal('2017-09-07T00:00:00-07:00', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#dateTime')))
(rdflib.term.BNode('f1454b393d3b34d8fbf97b594baecf658b7'), rdflib.term.Literal('p0-20170906t151820/messages-20170907'), rdflib.term.Literal('2017-09-07T00:00:01.008501-07:00', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#dateTime')))
(rdflib.term.BNode('f1454b393d3b34d8fbf97b594baecf658b3'), rdflib.term.Literal('p0-20170906t151820/console-20170907'), rdflib.term.Literal('2017-09-07T00:00:37.627889-07:00', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#dateTime')))


NameError: name '__version__' is not defined